In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

import polars as pl
from freezegun import freeze_time

from src.activities import (
    get_activities_df,
    get_activity_summaries,
    get_day_of_week_summaries,
    get_weekly_summaries,
)
from src.auth_manager import get_strava_client
from src.constants import COACH_ROLE
from src.email_manager import (
    send_email,
    training_week_to_html,
    training_week_update_to_html,
)
from src.supabase_client import (
    get_training_week_with_coaching,
    upsert_training_week_update,
    upsert_training_week_with_coaching,
)
from src.training_week import generate_training_week
from src.training_week_update import get_updated_training_week
from src.types.mid_week_analysis import MidWeekAnalysis


# Sunday Night Training Week Message

In [ ]:
client_preferences = "A) Training for a marathon B) This will be my second marathon C) Prefer workouts on Wednesdays and long runs on Saturdays"
sysmsg_base = f"{COACH_ROLE}\nYour client has included the following preferences: {client_preferences}\n"

# activities setup
athlete_id = os.environ["JAMIES_ATHLETE_ID"]
strava_client = get_strava_client(athlete_id)
activities_df = get_activities_df(strava_client)

# gen training week pipeline
day_of_week_summaries = get_day_of_week_summaries(activities_df)
weekly_summaries = get_weekly_summaries(activities_df)
training_week_with_coaching = generate_training_week(
    sysmsg_base=sysmsg_base,
    weekly_summaries=weekly_summaries,
    day_of_week_summaries=day_of_week_summaries,
)

# save data to db and trigger email
upsert_training_week_with_coaching(
    athlete_id=athlete_id, training_week_with_coaching=training_week_with_coaching
)
send_email(
    subject="Training Schedule Just Dropped 🏃",
    html_content=training_week_to_html(training_week_with_coaching),
)

# Mid-Week Update

In [ ]:
client_preferences = "A) Training for a marathon B) This will be my second marathon C) Prefer workouts on Wednesdays and long runs on Saturdays"
sysmsg_base = f"{COACH_ROLE}\nYour client has included the following preferences: {client_preferences}\n"

athlete_id = os.environ["JAMIES_ATHLETE_ID"]
strava_client = get_strava_client(athlete_id)

training_week_with_coaching = get_training_week_with_coaching(athlete_id)
current_weeks_activity_summaries = get_activity_summaries(strava_client, num_weeks=1)
mid_week_analysis = MidWeekAnalysis(
    activities=current_weeks_activity_summaries,
    training_week=training_week_with_coaching.training_week,
)
training_week_update_with_planning = get_updated_training_week(
    sysmsg_base=sysmsg_base, 
    mid_week_analysis=mid_week_analysis
)

upsert_training_week_update(
    athlete_id=athlete_id,
    mid_week_analysis=mid_week_analysis,
    training_week_update_with_planning=training_week_update_with_planning,
)

send_email(
    subject="Training Schedule Update 🏃",
    html_content=training_week_update_to_html(
        mid_week_analysis=mid_week_analysis, 
        training_week_update_with_planning=training_week_update_with_planning
    ),
)

# freeze time tests

In [120]:
from src.lambda_function import lambda_handler

@freeze_time("2024-08-18 20:00:00")
def test_sunday_workflow():
    lambda_handler({}, {})

@freeze_time("2024-08-20 20:00:00")
def test_mid_week_workflow():
    lambda_handler({}, {})

test_sunday_workflow()
test_mid_week_workflow()

athlete_id='98390356' token still valid until 2024-08-21 23:22:07+00:00
athlete_id='98390356' token still valid until 2024-08-21 23:22:07+00:00


In [5]:
from src.supabase_client import list_users

users = list_users()


[UserRow(athlete_id=98390356, email='voynow99@gmail.com', preferences='A) Training for a marathon B) This will be my second marathon C) Prefer workouts on Wednesdays and long runs on Saturdays', created_at=datetime.datetime(2024, 8, 21, 21, 12, 55, 723179, tzinfo=datetime.timezone.utc)),
 UserRow(athlete_id=104454087, email='Rachel.decker122@gmail.com', preferences='A) Training for a marathon B) This will be my first marathon C) Generally more concerned with staying healthy and building up my long run VS doing speed workouts', created_at=datetime.datetime(2024, 8, 21, 21, 14, 35, 885680, tzinfo=datetime.timezone.utc))]